# Patient Quality of Life Score Analysis

This notebook aggregates patient responses and calculates quality of life scores.

In [ ]:
import sqlite3
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for plots
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Connect to Database and Load Data

In [ ]:
# Connect to SQLite database
db_path = '../data/patient_portal.db'
conn = sqlite3.connect(db_path)

# Load patients data
patients_df = pd.read_sql_query("SELECT * FROM patients", conn)
print(f"Total patients: {len(patients_df)}")
patients_df.head()

In [ ]:
# Load responses data
responses_df = pd.read_sql_query("SELECT * FROM responses", conn)
print(f"Total responses: {len(responses_df)}")
responses_df.head()

In [ ]:
# Load quality of life scores
scores_df = pd.read_sql_query("""
    SELECT q.*, p.name, p.email, p.age
    FROM quality_of_life_scores q
    JOIN patients p ON q.patient_id = p.id
""", conn)
print(f"Total quality of life scores: {len(scores_df)}")
scores_df.head()

## 2. Parse Category Scores

In [ ]:
# Parse JSON category scores
def parse_category_scores(row):
    try:
        return json.loads(row['category_scores'])
    except:
        return {'physical': 0, 'mental': 0, 'social': 0}

scores_df['categories'] = scores_df.apply(parse_category_scores, axis=1)
scores_df['physical_score'] = scores_df['categories'].apply(lambda x: x.get('physical', 0))
scores_df['mental_score'] = scores_df['categories'].apply(lambda x: x.get('mental', 0))
scores_df['social_score'] = scores_df['categories'].apply(lambda x: x.get('social', 0))

scores_df[['name', 'total_score', 'physical_score', 'mental_score', 'social_score']].head()

## 3. Calculate Summary Statistics

In [ ]:
# Overall statistics
print("\n=== Quality of Life Score Statistics ===")
print(f"Mean Total Score: {scores_df['total_score'].mean():.2f}")
print(f"Median Total Score: {scores_df['total_score'].median():.2f}")
print(f"Std Dev: {scores_df['total_score'].std():.2f}")
print(f"Min Score: {scores_df['total_score'].min()}")
print(f"Max Score: {scores_df['total_score'].max()}")

print("\n=== Category Scores (Mean) ===")
print(f"Physical Health: {scores_df['physical_score'].mean():.2f}")
print(f"Mental Well-being: {scores_df['mental_score'].mean():.2f}")
print(f"Social Relationships: {scores_df['social_score'].mean():.2f}")

## 4. Visualize Quality of Life Scores

In [ ]:
# Distribution of total scores
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(scores_df['total_score'], bins=20, color='skyblue', edgecolor='black')
plt.xlabel('Total Quality of Life Score')
plt.ylabel('Frequency')
plt.title('Distribution of Quality of Life Scores')
plt.axvline(scores_df['total_score'].mean(), color='red', linestyle='--', label=f"Mean: {scores_df['total_score'].mean():.2f}")
plt.legend()

plt.subplot(1, 2, 2)
plt.boxplot(scores_df['total_score'])
plt.ylabel('Total Quality of Life Score')
plt.title('Box Plot of Quality of Life Scores')

plt.tight_layout()
plt.show()

In [ ]:
# Category scores comparison
category_means = {
    'Physical': scores_df['physical_score'].mean(),
    'Mental': scores_df['mental_score'].mean(),
    'Social': scores_df['social_score'].mean()
}

plt.figure(figsize=(10, 6))
plt.bar(category_means.keys(), category_means.values(), color=['#1976d2', '#7b1fa2', '#388e3c'])
plt.xlabel('Category')
plt.ylabel('Average Score')
plt.title('Average Quality of Life Scores by Category')
plt.ylim(0, max(category_means.values()) * 1.2)
for i, (k, v) in enumerate(category_means.items()):
    plt.text(i, v + 0.5, f'{v:.2f}', ha='center', va='bottom', fontweight='bold')
plt.show()

## 5. Age-Based Analysis

In [ ]:
# Create age groups
scores_df['age_group'] = pd.cut(scores_df['age'], 
                                  bins=[0, 30, 50, 70, 120], 
                                  labels=['18-30', '31-50', '51-70', '70+'])

# Group by age and calculate mean scores
age_analysis = scores_df.groupby('age_group').agg({
    'total_score': 'mean',
    'physical_score': 'mean',
    'mental_score': 'mean',
    'social_score': 'mean'
}).round(2)

print("\n=== Quality of Life by Age Group ===")
print(age_analysis)

In [ ]:
# Visualize age-based analysis
age_analysis.plot(kind='bar', figsize=(12, 6))
plt.xlabel('Age Group')
plt.ylabel('Average Score')
plt.title('Quality of Life Scores by Age Group and Category')
plt.legend(['Total', 'Physical', 'Mental', 'Social'])
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 6. Individual Patient Analysis

In [ ]:
# Function to analyze individual patient
def analyze_patient(patient_id):
    patient = scores_df[scores_df['patient_id'] == patient_id].iloc[0]
    
    print(f"\n=== Patient Analysis ===")
    print(f"Name: {patient['name']}")
    print(f"Age: {patient['age']}")
    print(f"\nScores:")
    print(f"  Total: {patient['total_score']}")
    print(f"  Physical: {patient['physical_score']}")
    print(f"  Mental: {patient['mental_score']}")
    print(f"  Social: {patient['social_score']}")
    
    # Radar chart
    categories = ['Physical', 'Mental', 'Social']
    values = [patient['physical_score'], patient['mental_score'], patient['social_score']]
    
    angles = [n / float(len(categories)) * 2 * 3.14159 for n in range(len(categories))]
    values += values[:1]
    angles += angles[:1]
    
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(projection='polar'))
    ax.plot(angles, values, 'o-', linewidth=2, label=patient['name'])
    ax.fill(angles, values, alpha=0.25)
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories)
    ax.set_ylim(0, 15)
    ax.set_title(f"Quality of Life Profile - {patient['name']}", size=16, pad=20)
    plt.legend(loc='upper right')
    plt.show()

# Example: analyze first patient (if exists)
if len(scores_df) > 0:
    analyze_patient(scores_df.iloc[0]['patient_id'])

## 7. Export Summary Report

In [ ]:
# Create summary report
summary_report = scores_df[['name', 'age', 'total_score', 'physical_score', 'mental_score', 'social_score', 'calculated_at']].copy()
summary_report.columns = ['Name', 'Age', 'Total Score', 'Physical', 'Mental', 'Social', 'Date']

# Save to CSV
output_path = '../data/quality_of_life_report.csv'
summary_report.to_csv(output_path, index=False)
print(f"\nSummary report saved to: {output_path}")
summary_report

In [ ]:
# Close database connection
conn.close()
print("\nDatabase connection closed.")